## Fábrica de Móveis

Uma grande fábrica de móveis dispõe em estoque de 250 metros de tábuas, 600 metros de pranchas e 500 metros de painéis de conglomerado. 

A fábrica normalmente oferece uma linha de móveis composta por um modelo de escrivaninha, uma mesa de reunião, um armário e uma prateleira. Cada tipo de móvel consome uma certa quantidade de matéria-prima, conforme a tabela a seguir. 

A escrivaninha é vendida por 100 unidades monetárias (u.m.), a mesa por 80 u.m., o armário por 120 u.m. e a prateleira por 20 u.m. 

Pede-se exibir um modelo de Programação Linear que maximize a receita com a venda dos móveis.

           Escrivania = x_1;          Mesa = x_2;          Armário = x_3;          Prateleira = x_4

       
Maximizar: 100x_1 + 80x_2 + 120x_3 + 20x_4

Sujeito a:

x_1 + x_2 + x_3 + 4x_4 <=250

x_2 + x_3 + 2x_4 <= 600

3x_1 + 2x_2 + 4x_3 <= 500

In [1]:
from docplex.mp.model import Model
import cplex

m = Model(name='Fabrica_De_Moveis')
x_1 = m.integer_var(name='x_1')
x_2 = m.integer_var(name='x_2')
x_3 = m.integer_var(name='x_3')
x_4 = m.integer_var(name='x_4')

m.add_constraint(x_1 + x_2 + x_3 + 4*x_4 <= 250)
m.add_constraint(x_2 + x_3 + 2*x_4 <= 600)
m.add_constraint(3*x_1 + 2*x_2 + 4*x_3 <= 500)
m.add_constraint(x_1 >= 0)
m.add_constraint(x_2 >= 0)
m.add_constraint(x_3 >= 0)
m.add_constraint(x_4 >= 0)

m.maximize(100*x_1 + 80*x_2 + 120*x_3 + 20*x_4)
m.solve()
print(m.solution)

solution for: Fabrica_De_Moveis
objective: 20000
x_2=250



### Usando o método de força bruta

1) testar, para cada combinação de m colunas de A, se elas formam uma base; 

2) calcular, para cada base, a solução básica associada a ela; 

3) calcular, para todas as SBV obtidas, o valor da função objetivo.

In [2]:
from math import factorial
from __future__ import print_function
import numpy as np
import pandas as pd

**Forma padrão:**

Maximizar: 100x_1 + 80x_2 + 120x_3 + 20x_4

Sujeito a:

x_1 + x_2 + x_3 + 4x_4 + x_5 = 250

x_2 + x_3 + 2x_4 + x6 = 600

3x_1 + 2x_2 + 4x_3 + x7 = 500

In [3]:
# Informar aqui os valores da matriz A
A = np.array([[1, 1, 1, 4, 1, 0, 0],
              [0, 1, 1, 2, 0, 1, 0],
              [3, 2, 4, 0, 0, 0, 1]])

In [4]:
# Informar aqui os valores da matriz b
b = np.array([[250], 
              [600], 
              [500]])

In [5]:
# Informar aqui os valores do vetor c
c = np.array([100, 80, 120, 20, 0, 0, 0])

In [6]:
m = len(A)
print('m:', m)

n = len(A[0])
print('n:', n)

combinacoes = factorial(n)/(factorial(m)*(factorial(n-m)))

print('O número máximo de soluções básicas:', combinacoes)

m: 3
n: 7
O número máximo de soluções básicas: 35.0


In [7]:
def verifica_solucao_viavel(x):
    '''
        Função que verifica se a solução é viável
        
        >>> verifica_solucao_viavel([0, 0, [125.], 0, [125.], [475.], 0])
            Esta é uma solução básica viável
            True
        >> verifica_solucao_viavel([0, 0, 0, [300.], [-950.], 0, [500.]])
            Esta NÃO é uma solução básica viável
            False
        >> verifica_solucao_viavel([1, 0, 1, 0, 1, 0, 0])
            Esta é uma solução básica viável
            True
    '''
    
    xB_bool = []

    for w in range(0, len(x)):
        if(x[w] >= 0):
            xB_bool.append(True)           
        else:
            xB_bool.append(False)

        try:
            if(x[w][0] >= 0):
                xB_bool.append(True)
            else:
                xB_bool.append(False)
        except:
            pass
            
    if(all(xB_bool)):
        print('Esta é uma solução básica viável')
        return True
    else:
        print('Esta NÃO é uma solução básica viável')
        return False

In [8]:
solucoes_viaveis = np.array([[0,0,0], 0,0,0], dtype=object)
todas_solucoes = np.array([[0,0,0], 0,0,0], dtype=object)
cont_x = 0

for w in range(0, n):
    for y in range(w+1, n):
        for z in range(y+1, n):
            
            # Id da solução
            cont_x = cont_x + 1;
            print('\nx', cont_x)
            
            # Conjunto de índices IB
            IB = [w,y,z]
            print('IB =', IB[0]+1, IB[1]+1, IB[2]+1)
            
            # B
            B = np.array([A[0][w], A[0][y], A[0][z]])
            for v in range (1, m):
                B = np.row_stack(tup=(B, [A[v][w], A[v][y], A[v][z]]))
            print('B =', B, '\n')
            
            # Verificando se B é uma base
            v, V =  np.linalg.eig(B)
            
            if(v.all() == 0):
                print('A matriz B não é uma base')
                #print('A linha LD é:')
                #print (B[v == 0,:])
                print('\n---------------------------------------------------------------------------\n')
                
                # Colocar na tabela de soluções básicas, com informações vazias
                todas_solucoes = np.row_stack((todas_solucoes, [[IB[0]+1, IB[1]+1, IB[2]+1], cont_x, [], []]))
                continue

            # B-1
            try:
                B1 = np.linalg.inv(B)
                print('B-1 =', B1, '\n')
            except:
                print("B-1 não existe")
                todas_solucoes = np.row_stack((todas_solucoes, [[IB[0]+1, IB[1]+1, IB[2]+1], cont_x, [], []]))
                print('\n---------------------------------------------------------------------------\n')
                continue

            # xB
            xB = B1.dot(b)
            print('xB =', xB, '\n')
            
            # x
            x = []
            for v in range (0, n):              
                if(v == IB[0]):
                    x.append(np.round(xB[0]))
                elif(v == IB[1]):
                    x.append(np.round(xB[1]))
                elif(v == IB[2]):
                    x.append(np.round(xB[2]))
                else:
                    x.append(0)
            print('x =', x, '\n')
                
            # cTx
            cTx = np.transpose(c).dot(x)
            print('cTx =', cTx, '\n')
                
            # Colocar na tabela de soluções básicas
            todas_solucoes = np.row_stack((todas_solucoes, [[IB[0]+1, IB[1]+1, IB[2]+1], cont_x, cTx, x]))
            
            # Se a solução for viável, colocar na tabela de soluções viáveis
            if(verifica_solucao_viavel(x) == True):
                solucoes_viaveis = np.row_stack((solucoes_viaveis, [[IB[0]+1, IB[1]+1, IB[2]+1], cont_x, cTx, x]))
            
            print('\n---------------------------------------------------------------------------\n')



x 1
IB = 1 2 3
B = [[1 1 1]
 [0 1 1]
 [3 2 4]] 

B-1 = [[ 1.  -1.   0. ]
 [ 1.5  0.5 -0.5]
 [-1.5  0.5  0.5]] 

xB = [[-350.]
 [ 425.]
 [ 175.]] 

x = [array([-350.]), array([425.]), array([175.]), 0, 0, 0, 0] 

cTx = [20000.] 

Esta NÃO é uma solução básica viável

---------------------------------------------------------------------------


x 2
IB = 1 2 4
B = [[1 1 4]
 [0 1 2]
 [3 2 0]] 

B-1 = [[ 0.4 -0.8  0.2]
 [-0.6  1.2  0.2]
 [ 0.3 -0.1 -0.1]] 

xB = [[-280.]
 [ 670.]
 [ -35.]] 

x = [array([-280.]), array([670.]), 0, array([-35.]), 0, 0, 0] 

cTx = [24900.] 

Esta NÃO é uma solução básica viável

---------------------------------------------------------------------------


x 3
IB = 1 2 5
B = [[1 1 1]
 [0 1 0]
 [3 2 0]] 

B-1 = [[ 0.         -0.66666667  0.33333333]
 [ 0.          1.          0.        ]
 [ 1.         -0.33333333 -0.33333333]] 

xB = [[-233.33333333]
 [ 600.        ]
 [-116.66666667]] 

x = [array([-233.]), array([600.]), 0, 0, array([-117.]), 0, 0] 

cTx = [24

<ipython-input-8-ebfe8f59c227>:64: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  cTx = np.transpose(c).dot(x)
/home/myllah/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


In [9]:
## Formatando os valores do todas_solucoes para sair bonitinho na tabela

for x in range(1, len(todas_solucoes)):
    try:
        todas_solucoes[x][2] = todas_solucoes[x][2][0]
    except:
        pass

for x in range(1, len(todas_solucoes)):
    for y in range(0, len(todas_solucoes)):
        try:
            todas_solucoes[x][3][y] = todas_solucoes[x][3][y][0]
        except:
            pass

In [10]:
print("Todas as soluções:")

todas_solucoes = pd.DataFrame(todas_solucoes[1:,:], columns=['Conjunto de índices base', 'Id da solução', 'cTx', 'Solução básica associada à base'])
todas_solucoes.sort_values(by=['Id da solução'])

todas_solucoes

Todas as soluções:


,Conjunto de índices base,Id da solução,cTx,Solução básica associada à base
0,"[1, 2, 3]",1,20000.0,"[-350.0, 425.0, 175.0, 0, 0, 0, 0]"
1,"[1, 2, 4]",2,24900.0,"[-280.0, 670.0, 0, -35.0, 0, 0, 0]"
2,"[1, 2, 5]",3,24700.0,"[-233.0, 600.0, 0, 0, -117.0, 0, 0]"
3,"[1, 2, 6]",4,20000.0,"[0.0, 250.0, 0, 0, 0, 350.0, 0]"
4,"[1, 2, 7]",5,13000.0,"[-350.0, 600.0, 0, 0, 0, 0, 350.0]"
5,"[1, 3, 4]",6,11600.0,"[-471.0, 0, 479.0, 61.0, 0, 0, 0]"
6,"[1, 3, 5]",7,8700.0,"[-633.0, 0, 600.0, 0, 283.0, 0, 0]"
7,"[1, 3, 6]",8,20000.0,"[500.0, 0, -250.0, 0, 0, 850.0, 0]"
8,"[1, 3, 7]",9,37000.0,"[-350.0, 0, 600.0, 0, 0, 0, -850.0]"
9,"[1, 4, 5]",10,22700.0,"[167.0, 0, 0, 300.0, -1117.0, 0, 0]"


In [11]:
qtd_solucoes_basicas = list(map(lambda z, sb=0: sb if z == [] else sb+1, todas_solucoes.loc[:, 'cTx'])).count(1)

print("\nQuantidade de soluções básicas: ", qtd_solucoes_basicas)


Quantidade de soluções básicas:  32


<ipython-input-11-243826ed9650>:1: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  qtd_solucoes_basicas = list(map(lambda z, sb=0: sb if z == [] else sb+1, todas_solucoes.loc[:, 'cTx'])).count(1)


In [12]:
## Formatando os valores do solucoes_viaveis para sair bonitinho na tabela

for x in range(1, len(solucoes_viaveis)):
    try:
        solucoes_viaveis[x][2] = solucoes_viaveis[x][2][0]
    except:
        pass

for x in range(1, len(solucoes_viaveis)):
    for y in range(0, len(solucoes_viaveis)):
        try:
            solucoes_viaveis[x][3][y] = solucoes_viaveis[x][3][y][0]
        except:
            pass

In [13]:
print("Soluções viáveis:")

solucoes_viaveis = pd.DataFrame(solucoes_viaveis[1:,:], columns=['Conjunto de índices base', 'Id da solução', 'cTx', 'Solução básica associada à base'])
solucoes_viaveis.sort_values(by=['Id da solução'])

solucoes_viaveis

Soluções viáveis:


,Conjunto de índices base,Id da solução,cTx,Solução básica associada à base
0,"[1, 2, 6]",4,20000.0,"[0.0, 250.0, 0, 0, 0, 350.0, 0]"
1,"[1, 4, 6]",11,17120.0,"[167.0, 0, 0, 21.0, 0, 558.0, 0]"
2,"[1, 5, 6]",13,16700.0,"[167.0, 0, 0, 0, 83.0, 600.0, 0]"
3,"[2, 3, 6]",18,20000.0,"[0, 250.0, 0.0, 0, 0, 350.0, 0]"
4,"[2, 4, 6]",21,20000.0,"[0, 250.0, 0, 0.0, 0, 350.0, 0]"
5,"[2, 5, 6]",23,20000.0,"[0, 250.0, 0, 0, 0.0, 350.0, 0]"
6,"[2, 6, 7]",25,20000.0,"[0, 250.0, 0, 0, 0, 350.0, 0.0]"
7,"[3, 4, 6]",27,15620.0,"[0, 0, 125.0, 31.0, 0, 412.0, 0]"
8,"[3, 5, 6]",29,15000.0,"[0, 0, 125.0, 0, 125.0, 475.0, 0]"
9,"[4, 6, 7]",34,1240.0,"[0, 0, 0, 62.0, 0, 475.0, 500.0]"


In [14]:
qtd_solucoes_viaveis = len(solucoes_viaveis)
print("\nQuantidade de soluções viáveis: ", qtd_solucoes_viaveis)


Quantidade de soluções viáveis:  11


In [15]:
## Digite 1 se você deseja maximizar ou 2 se você deseja minimizar
entrada = 1

if(entrada == 1):
    id_solucao_otima = solucoes_viaveis['cTx'].astype(float).argmax()
elif(entrada == 2):
    id_solucao_otima = solucoes_viaveis['cTx'].astype(float).argmin()
    
solucao_otima = solucoes_viaveis['cTx'][id_solucao_otima]
geram_solucoes_otimas =  solucoes_viaveis['cTx'][id_solucao_otima]
print('Solução Ótima:', solucao_otima)

Solução Ótima: 20000.0


In [16]:
print('\nSoluções que geram a solução ótima:\n')

geram_solucoes_otimas = solucoes_viaveis.loc[solucoes_viaveis['cTx'] == solucao_otima, :]

geram_solucoes_otimas


Soluções que geram a solução ótima:



,Conjunto de índices base,Id da solução,cTx,Solução básica associada à base
0,"[1, 2, 6]",4,20000.0,"[0.0, 250.0, 0, 0, 0, 350.0, 0]"
3,"[2, 3, 6]",18,20000.0,"[0, 250.0, 0.0, 0, 0, 350.0, 0]"
4,"[2, 4, 6]",21,20000.0,"[0, 250.0, 0, 0.0, 0, 350.0, 0]"
5,"[2, 5, 6]",23,20000.0,"[0, 250.0, 0, 0, 0.0, 350.0, 0]"
6,"[2, 6, 7]",25,20000.0,"[0, 250.0, 0, 0, 0, 350.0, 0.0]"


In [17]:
print('\nO número máximo de soluções básicas:', combinacoes)
print("\nQuantidade de soluções básicas: ", qtd_solucoes_basicas)
print("\nQuantidade de soluções viáveis: ", qtd_solucoes_viaveis)
print('\nSolução Ótima:', solucao_otima)


O número máximo de soluções básicas: 35.0

Quantidade de soluções básicas:  32

Quantidade de soluções viáveis:  11

Solução Ótima: 20000.0
